Hello, my name is Artem. I'm going to review your project!

You can find my comments in <font color='green'>green</font>, <font color='blue'>blue</font> or <font color='red'>red</font> boxes like this:

<div class="alert alert-block alert-success">
<b>Success:</b> if everything is done succesfully
</div>

<div class="alert alert-block alert-info">
<b>Improve: </b> "Improve" comments mean that there are tiny corrections that could help you to make your project better.
</div>

<div class="alert alert-block alert-danger">
<b>Needs fixing:</b> if the block requires some corrections. Work can't be accepted with the red comments.
</div>

### <font color='orange'>General feedback</font>
* You've worked really hard and submitted a solid project.
* Thank you for structuring the project. It's a pleasure to check such notebooks.
* There are a couple of things that need to be done before your project is complete, but they're pretty straightforward.
* While there's room for improvement, on the whole, your project is looking good.
* I believe you can easily fix it! Good luck!

**Comment:** Section 4 and onward has been redone with a different bootstrapping method. Please recheck.

### <font color='orange'>General feedback (review 2)</font>
* New comments are marked with "review 2" keyword.

### <font color='orange'>General feedback (review 3)</font>
* Sorry but I don't see any changes.
* You are free to ask some questions if needed.

**Comment:** There must have been some technical issue, there was an issue I had with JupyterHub last night as I was submitting. Since both the feedback and my changes got erased. Since Section 4 (calculating profits) is the section requiring feedback, that is the one where the changes are.

### <font color='orange'>General feedback (review 4)</font>
* All your hard work has paid off, and now your project is perfect!
* New comments are marked with "review 4" keyword.
* Keep up the good work. Good luck next!

## Oil Well Project

The purpose of this project is to use machine learning to determine the best location to create a new oil well. We will pick the 200 most profitable points, and in order to determine profit we will take the following assumptions:

* The budget for development of 200 oil wells is 100 million USD.

* One barrel of raw materials brings 4.5 USD of revenue. One unit of product is defined as 1000 barrels. 

* Thus, the revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels)

* We wish to keep only the regions with the risk of losses lower than 2.5%.

* The only suitable model is known to be linear regression

* All mentioned dollar amounts are in thousands of dollars

## Initialization

In [1]:
# load packages

import pandas as pd
import numpy as np
from scipy import stats as st
from statistics import mean

# model creation

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# model checking

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

<div class="alert alert-block alert-success">
<b>Success:</b> Thank you for collecting all imports in the first cell!
</div>

In [2]:
# import data

geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

After importing our data, we will take a quick look at what our data looks like.

In [3]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


We were informed beforehand that the data possesses the ID, three points of interest that have been abstracted for us, and the amount of product that the well produces in thousands of barrels. Based on the info scans, all our data has been cleanly imported for us.

We can get a glance at the differences between our data, based on the descriptive statistics.

In [7]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [8]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [9]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


It appears that the variables are fairly different across the models. f0 is the most different and clearly not the same across the three data sets. f1 also has a wide range of values. f2 has a similar mean across the three data sets, but it appears to have some large outliers. The product category is a bit similar. Regardless, we can't treat our data as being the same.

<div class="alert alert-block alert-success">
<b>Success:</b> Data loading and initial analysis were done well!
</div>

## Testing model per region

First, we will create a linear regression model for each region. In order to do so, first we should split each of our data sets into a features and target data, and establish the size of our test set (25% of our data).

In [10]:
# set features and target

features_0 = geo_data_0.drop(['id', 'product'], axis=1)
target_0 = geo_data_0['product']

features_1 = geo_data_1.drop(['id', 'product'], axis=1)
target_1 = geo_data_1['product']

features_2 = geo_data_2.drop(['id', 'product'], axis=1)
target_2 = geo_data_2['product']

# set size of our training/test split - we will keep it simple and use a standard 75-25 split

size = 0.25

<div class="alert alert-block alert-success">
<b>Success:</b> I agree that we don't need "id" column.
</div>

We will create a function that will give us information about training and testing sets that we can reference anywhere we might need it.

In [11]:
def print_train_test(features, target):
    features_train, features_test, target_train, target_test = train_test_split(features, target,
        test_size = size, random_state = 12345)
    
    print("Training features shape:", features_train.shape)
    print("Validation features shape:", features_test.shape)

In [12]:
print_train_test(features_0, target_0)
print()
print_train_test(features_1, target_1)
print()
print_train_test(features_2, target_2)

Training features shape: (75000, 3)
Validation features shape: (25000, 3)

Training features shape: (75000, 3)
Validation features shape: (25000, 3)

Training features shape: (75000, 3)
Validation features shape: (25000, 3)


We see all our data has been split appropriately with this function. Then we can make a function that will create a model and return our predictions and correct answers that we can run across every data set.

In [13]:
def get_predicted_valid(features, target):
    
    features_train, features_test, target_train, target_test = train_test_split(features, target,
        test_size = size, random_state = 12345)
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    
    features_train = scaler.transform(features_train)
    features_test = scaler.transform(features_test)
    
    # create model
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_test)
    
    return predicted_valid, target_test

<div class="alert alert-block alert-success">
<b>Success:</b> Great that scaler was fitted only on train set.
</div>

In [14]:
get_predicted_valid(features_0, target_0)

(array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
        118.18039721, 118.16939229]),
 71751     10.038645
 80493    114.551489
 2655     132.603635
 53233    169.072125
 91141    122.325180
             ...    
 12581    170.116726
 18456     93.632175
 73035    127.352259
 63834     99.782700
 43558    177.821022
 Name: product, Length: 25000, dtype: float64)

This confirms that the function will work, now we can run it across all of our data sets and get our predictions.

In [15]:
# region zero

pred_results_0, target_results_0 = get_predicted_valid(features_0, target_0)

print("Area 0:\n")
print("Mean of predictions:", pred_results_0.mean())
print("Mean of target:", target_results_0.mean())

print("Root Mean squared error:", mean_squared_error(target_results_0, pred_results_0) ** 0.5)

Area 0:

Mean of predictions: 92.59256778438035
Mean of target: 92.07859674082927
Root Mean squared error: 37.5794217150813


In [16]:
# region one

pred_results_1, target_results_1 = get_predicted_valid(features_1, target_1)

print("Area 1:\n")
print("Mean of predictions:", pred_results_1.mean())
print("Mean of target:", target_results_1.mean())

print("Root Mean squared error:", mean_squared_error(target_results_1, pred_results_1) ** 0.5)

Area 1:

Mean of predictions: 68.728546895446
Mean of target: 68.72313602435997
Root Mean squared error: 0.893099286775617


In [17]:
# region two

pred_results_2, target_results_2 = get_predicted_valid(features_2, target_2)

print("Area 2:\n")
print("Mean of predictions:", pred_results_2.mean())
print("Mean of target:", target_results_2.mean())

print("Root Mean squared error:", mean_squared_error(target_results_2, pred_results_2) ** 0.5)

Area 2:

Mean of predictions: 94.96504596800489
Mean of target: 94.88423280885438
Root Mean squared error: 40.02970873393434


Based on our results, the models appear well fit. The root mean square error is similar for areas zero and two, but quite different for area one. Based on the descriptive statistics, it is possible that means that the predictor f2 has a large influence on our error rate, as it is the only predictor that is fairly similar between data sets zero and two (being 3.2482 and 3.4734, respectively) but is clearly different for data set one (1.7036).

However, it is also possible that the mean square error being largely different is a result of the other two predictors, or also it could be a result of the targets being different. At any rate, we can say that 

<div class="alert alert-block alert-success">
<b>Success:</b> Models were trained correctly. Glad to see that you've avoid code duplication.
</div>

## Calculate possible profits

As stated before, we will first initialize some of our key variables to work with on our financials.

One key thing to keep in mind is that our 'product' variable is dealing in thousands. We could solve this in one of two ways: either we adjust the variable or we adjust all our other data to be done in thousands. We will opt for the latter, and this choice is stated in section 1.

<div class="alert alert-block alert-danger">

<b>Needs fixing:</b> Let me explain algorithm one more time:
1. From previous steps we get 25000 predicted and 25000 real values for each region.
2. In bootstrap, we randomly select 500 points out of 25000 and pass them into function for profit calculation.
    1. This function select best 200 wells by predicted values.
    2. After that we calculate revenue with the help of real values of <b>corresponding</b> wells.
    3. At the end of this function we calculate `profit=revenue-costs`.
3. We run 1000 times algorithm that is described above [each time sample 500 random wells out of 25000] and add all values [i mean result of "profit" function] to list (e.g. `profit_list`).
4. After that we calculate mean of this list, risk of losses (percentage of negative values in this list) and confidence interval (just 0.025 and 0.975 quantiles of this list).
</div>

**Comment:** Section has been mostly redone, please recheck.

In [18]:
# budget is $100,000,000. Since our numbers deal in thousands, we should adjust that to 100,000 sets of $1000

budget = 100000

# the gross profit for unit of production - since we are dealing in thousands, 1 unit = $4500 = 4.5

gross_profit_per_unit = 4.5

# set number of wells

wells = 200
cost_per_well = budget / wells

# find minimum revenue

minimum_product = cost_per_well / gross_profit_per_unit

# set random state

state = np.random.RandomState(12345)

# we can initialize a variable to equal our threshold for loss amount

loss_threshold = 0.025

<div class="alert alert-block alert-info">
<b>Improve (review 2): </b> It would be better if only uppercase letters were used in constant variable names.
</div>

In [19]:
print("In order to break even, a well must produce at least", minimum_product, "units of product.")
print("To break even, the total production of our 200 wells should be", minimum_product * wells, "units of product.")

In order to break even, a well must produce at least 111.11111111111111 units of product.
To break even, the total production of our 200 wells should be 22222.222222222223 units of product.


<div class="alert alert-block alert-success">
<b>Success (review 2):</b> The breakeven point was found correctly!
</div>

We know the average number of units a single well should produce, as well as the total value needed across all wells (this will be useful since it is certain that each well will produce a different amount of product. Now, we need to make a function that will let us access the predictions of our top 200 wells. To do this, we can simply sort our predictions from greatest to lowest and then limit that list to the top 200 wells, then add together the predicted units of product that each

In [20]:
# a function that will return the sum of the production for the top 200 wells

def top_wells_prediction(prediction, real):
    prediction_sorted = prediction.sort_values(ascending = False)
    top_wells_prediction = real[prediction_sorted.index][:wells]
    return top_wells_prediction.sum()

# a function to find the profit, given the total production

def find_profit(production, budget):
    revenue = production * gross_profit_per_unit
    profit = revenue - budget
    
    # convert into dollar amount, since profit is in thousands
    converted_profit = profit * 1000
    return converted_profit

In [32]:
def bootstrap_test(probabilities, target):
    production_list = []
    probabilities = pd.Series(probabilities, index = target.index)

    for i in range(1000):
        # get target/prob
        target_subsample = target.sample(n = 500, replace = True, random_state = state)
        probs_subsample = probabilities[target_subsample.index]
    
        # get avg revenue
        res_subsample = top_wells_prediction(probs_subsample, target_subsample)
    
        # append values
        production_list.append(res_subsample)
    
    # get list of production and profit
    
    production_list = pd.Series(production_list)

    mean = production_list.mean()
    
    profit_list = find_profit(production_list, budget)
    profit_mean = profit_list.mean()
    
    # create 95% CI for profits
    
    ci_lower = profit_list.quantile(0.025)
    ci_upper = profit_list.quantile(0.975)
    
    # calculate risk of loss
    risk_chance = (len([value for value in profit_list if value < 0]) / len(profit_list)) * 100
    
    print("Average production:", mean, "units")
    print()
    print("Average profit: $", profit_mean)
    print("Average profit 95% CI:", "(", ci_lower, ",", ci_upper, ")")
    print()
    print("Risk of loss:", risk_chance, "%")
    if (risk_chance / 100) > loss_threshold:
        print("Risk of loss too high.")

<div class="alert alert-block alert-danger">

<b>Needs fixing (review 2):</b> 
1. `state` is not defined
2. Parameters in `top_wells_prediction` function were passed in incorrect order.
3. There is no need to use 0.01 or 0.99 quantilies.
4. Confidence interval should be calculated in the function above (no need for separate function). Just 0.025 and 0.975 quantilies of `profit_list`.
5. Risk of losses = percent of negative values in `profit_list`.
</div>

**Comments:** This is the section that was altered. The issues 1-3 were cleared out (state is defined in the variable definition at the start of the section). Calculating confidence interval and risk of loss were added to the function.

In [33]:
print("Area Zero:")
bootstrap_test(pred_results_0, target_results_0)

Area Zero:
Average production: 23146.0660712566 units

Average profit: $ 4157297.3206547024
Average profit 95% CI: ( -839040.0418059878 , 9672671.737710377 )

Risk of loss: 5.7 %
Risk of loss too high.


In [34]:
print("Area One:")
bootstrap_test(pred_results_1, target_results_1)

Area One:
Average production: 23364.217316883412 units

Average profit: $ 5138977.925975348
Average profit 95% CI: ( 880463.2393889854 , 9559672.457516246 )

Risk of loss: 1.0999999999999999 %


In [35]:
print("Area Two:")
bootstrap_test(pred_results_2, target_results_2)

Area Two:
Average production: 23169.050792875358 units

Average profit: $ 4260728.56793911
Average profit 95% CI: ( -1429805.6483970576 , 9502194.53489145 )

Risk of loss: 7.5 %
Risk of loss too high.


Based on the bootstrapped model, it seems that Area One is the only viable option. While each area does reach an average production that surpasses our budget and thus they all have an average that is above our threshold of 22222 units of product. However, areas zero and two also have a risk of losses in their 95% confidence intervals. This risk is calculated as above our threshold of 2.5%, which means we cannot consider either to be an option and thus area one is the area that will be chosen to drill.

<div class="alert alert-block alert-success">
<b>Success (review 4):</b> All statistics are calculated correctly now!
</div>

## Conclusion

In conclusion, we had analyzed our three models using linear regression in order to determine which one had the highest profits. From our model testing, we would too at risk of loss in areas zero and two, but in area one we would be able to profit.

<div class="alert alert-block alert-success">
<b>Success (review 4):</b> I agree with your choice!
</div>